In [1]:
import pandas as pd
import numpy as np
import re
import datetime

### Zaimportuj i przejrzyj dane

In [2]:
dane = pd.read_csv('DataWorkshop.csv')

In [3]:
pd.options.display.max_columns=30

dane.head()

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,Skrzyżowanie,Geometria drogi,Charakterystyka miejsca zdarzenia,Oświetlenie,Warunki atmosferyczne,Rodzaj zdarzenia,Wina,Rodzaj drogi,Nawierzchnia drogi,Stan nawierzchni,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
0,2016-01-01,05:15:00,18*38'089,50*22'110,GLIWICE,TOSZECKA,NaN,901,"61,6",Obszar niezabudowany,NaN,Odcinek prosty,Jezdnia,Noc - droga nieoświetlona,Dobre warunki atmosferyczne,Najechanie na zwierzę,z innych przyczyn,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0
1,2016-01-01,00:05:00,19*10'510,50*19'231,DĄBROWA GÓRNICZA,SIENKIEWICZA,NaN,G200 030S,NaN,Obszar zabudowany,Z drogą z pierwsz.,NaN,Jezdnia,Noc - droga oświetlona,Dobre warunki atmosferyczne,Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,NaN,Nieprawidłowe: cofanie,NaN,NaN,0,0,1,0
2,2016-01-01,18:30:00,18*52'395,50*26'423,TARNOWSKIE GÓRY,KS.IGNACEGO SIWCA,4A,GMINNA,NaN,Obszar zabudowany,NaN,"Zakręt, łuk",Jezdnia,Noc - droga oświetlona,Dobre warunki atmosferyczne,Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Nie ma,2.0,Nieprawidłowe: omijanie,NaN,NaN,0,0,1,0
3,2016-01-01,15:10:00,18*51'381,50*26'408,TARNOWSKIE GÓRY,JÓZEFA PIŁSUDSKIEGO,6,P3276S,NaN,Obszar zabudowany,NaN,Odcinek prosty,Jezdnia,Światło dzienne,Pochmurno,Zderzenie pojazdów tylne,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,2.0,Nieprawidłowe: omijanie,NaN,NaN,0,0,1,0
4,2016-01-01,17:45:00,19*02'512,50*28'005,OŻAROWICE,DWORCOWA,16,P3203S,NaN,Obszar zabudowany,NaN,Odcinek prosty,Jezdnia,"Zmrok, świt",Dobre warunki atmosferyczne,Najechanie na zwierzę,z innych przyczyn,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Nie ma,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0


In [4]:
dane.shape

(229929, 30)

In [5]:
dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229929 entries, 0 to 229928
Data columns (total 30 columns):
Data zdarzenia                       229929 non-null object
Godzina                              229929 non-null object
GPS x                                229929 non-null object
GPS y                                229929 non-null object
Miejscowość                          229929 non-null object
Ulica                                209201 non-null object
Numer domu                           136045 non-null object
Numer drogi                          229929 non-null object
KM HM (Pikietaż)                     35041 non-null object
Obszar                               229928 non-null object
Skrzyżowanie                         63150 non-null object
Geometria drogi                      163921 non-null object
Charakterystyka miejsca zdarzenia    229927 non-null object
Oświetlenie                          229929 non-null object
Warunki atmosferyczne                229928 non-nul

In [6]:
print('Puste komórki:', pd.isnull(dane).sum().sum())
print('Wszystkie komórki:',229929*30)

Puste komórki: 1089521
Wszystkie komórki: 6897870


In [7]:
print('Puste komórki jako procent zbioru:',1089521*100//6897870, '%')


Puste komórki jako procent zbioru: 15 %


In [8]:
dane.dtypes

Data zdarzenia                        object
Godzina                               object
GPS x                                 object
GPS y                                 object
Miejscowość                           object
Ulica                                 object
Numer domu                            object
Numer drogi                           object
KM HM (Pikietaż)                      object
Obszar                                object
Skrzyżowanie                          object
Geometria drogi                       object
Charakterystyka miejsca zdarzenia     object
Oświetlenie                           object
Warunki atmosferyczne                 object
Rodzaj zdarzenia                      object
Wina                                  object
Rodzaj drogi                          object
Nawierzchnia drogi                    object
Stan nawierzchni                      object
Sygnalizacja świetlna                 object
Oznakowanie poziome                   object
Liczba pas

In [9]:
dane.sample(3)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,Skrzyżowanie,Geometria drogi,Charakterystyka miejsca zdarzenia,Oświetlenie,Warunki atmosferyczne,Rodzaj zdarzenia,Wina,Rodzaj drogi,Nawierzchnia drogi,Stan nawierzchni,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
37749,2016-09-30,16:30:00,18*46'319,50*18'445,ZABRZE,JANA KOCHANOWSKIEGO,19,G145458S,NaN,Obszar zabudowany,NaN,Odcinek prosty,Jezdnia,Światło dzienne,Dobre warunki atmosferyczne,Zderzenie pojazdów tylne,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Nie ma,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0
171179,2019-01-15,13:02:00,18*52'331,50*20'540,BYTOM,KONSTYTUCJI,85,P8613S,NaN,Obszar zabudowany,NaN,Odcinek prosty,Jezdnia,Światło dzienne,"Pochmurno; Opady śniegu, gradu",Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,"Oblodzenie, zaśnieżenie",Brak,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0
120498,2018-03-06,16:33:00,19*19'462,50*34'319,MYSZKÓW,1 MAJA,1,I2,NaN,Obszar zabudowany,NaN,Odcinek prosty,"Parking, plac, MOP",Światło dzienne,Dobre warunki atmosferyczne,Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0


### Wyczyść dane

#### Konwersja typu danych.

In [10]:
df = dane.copy()

In [11]:
#zmieńmy nazwy kolumn na prostsze/bez polskich zanków/bez spacji

nowe_nazwy = {'Data zdarzenia':'data_zdarzenia', 
              'Godzina':'godzina', 
              'GPS x':'GPS_x', 
              'GPS y':'GPS_y', 
              'Miejscowość':'miejscowosc', 
              'Ulica':'ulica',
              'Numer domu':'numer_domu', 
              'Numer drogi':'numer_drogi',
              'KM HM (Pikietaż)':'pikietaz', 
              'Obszar':'obszar',
              'Skrzyżowanie':'skrzyzowanie', 
              'Geometria drogi':'geometria_drogi',
              'Charakterystyka miejsca zdarzenia':'charakterystyka_miejsca',
              'Oświetlenie':'oswietlenie', 
              'Warunki atmosferyczne':'warunki_atmosferyczne',
              'Rodzaj zdarzenia':'rodzaj_zdarzenia', 
              'Wina':'wina',
              'Rodzaj drogi':'rodzaj_drogi',
              'Nawierzchnia drogi':'nawierzchnia_drogi',
              'Stan nawierzchni':'stan_nawierzchni',
              'Sygnalizacja świetlna':'sygnalizacja',
              'Oznakowanie poziome':'oznakowanie_poziome',
              'Liczba pasów w zdarzeniu':'liczba_pasow',
              'Zachowanie kierującego':'zachowanie_kierujacego',
              'Zachowanie pieszego':'zachowanie_pieszego',
              'Inne przyczyny':'inne_przyczyny',
              'Liczba rannych w zdarzeniu':'liczba_rannych',
              'Liczba zabitych w zdarzeniu':'liczba_zabitych',
              'Liczba kolizji':'liczba_kolizji', 
              'Liczba wypadków':'liczba_wypadkow'}
df.rename(columns=nowe_nazwy, inplace=True)

#### Kolumny zawierające datę i czas

In [12]:
print('kolumna DATA ZDARZENIA: ', df['data_zdarzenia'].dtypes)
print('kolumna GODZINA: ', df['data_zdarzenia'].dtypes)
print('Potrzebna konwersja do typu DATETIME')

kolumna DATA ZDARZENIA:  object
kolumna GODZINA:  object
Potrzebna konwersja do typu DATETIME


In [13]:
# Przekonwertujmy do formatu datetime
#Create additional TIME columns so it would be easier to manipulate the data

df['data_zdarzenia'] = pd.to_datetime(df['data_zdarzenia'])

df['godzina'] = pd.to_datetime(df['godzina'])

df['rok'] = pd.DatetimeIndex(df['data_zdarzenia']).year

df['miesiac'] = pd.DatetimeIndex(df['data_zdarzenia']).month

df['dzien_miesiąca'] = pd.DatetimeIndex(df['data_zdarzenia']).day

df['dzien_tygodnia'] = pd.DatetimeIndex(df['data_zdarzenia']).dayofweek

df['data'] = pd.DatetimeIndex(df['data_zdarzenia']).date

df['godzina'] = pd.DatetimeIndex(df['godzina']).hour

#Czy to był weekend?
df['czy_weekend'] = df['dzien_tygodnia'].apply(lambda x: 1 if (x == 6 or x == 5 ) else 0 )

#czy to była niedziela handlowa?
#niedziele handlowe w 2018 i 2019 roku:

nh = [datetime.date(2018,3,4),
        datetime.date(2018,3,25),
        datetime.date(2018,5,6),
        datetime.date(2018,5,27),
        datetime.date(2018,6,3),
        datetime.date(2018,6,24),
        datetime.date(2018,7,1),
        datetime.date(2018,7,29),
        datetime.date(2018,8,5),
        datetime.date(2018,8,26),
        datetime.date(2018,9,2),
        datetime.date(2018,9,30),
        datetime.date(2018,10,7),
        datetime.date(2018,10,28),
        datetime.date(2018,11,4),
        datetime.date(2018,11,25),
        datetime.date(2018,12,2),
        datetime.date(2018,12,16),
        datetime.date(2018,12,23),
        datetime.date(2018,12,30),
        datetime.date(2019,1,27),
        datetime.date(2019,2,24),
        datetime.date(2019,3,31),
        datetime.date(2019,4,14),
        datetime.date(2019,4,28),
        datetime.date(2019,5,26),
        datetime.date(2019,6,30),
        datetime.date(2019,7,28),
        datetime.date(2019,8,25),
        datetime.date(2019,9,29),
        datetime.date(2019,10,27),
        datetime.date(2019,11,24),
        datetime.date(2019,12,15),
        datetime.date(2019,12,22),
        datetime.date(2019,12,29),]

df['handlowa'] = df['data'].apply(lambda x: 1 if (x in nh) else 0 )

df.sample(2)

,data_zdarzenia,godzina,GPS_x,GPS_y,miejscowosc,ulica,numer_domu,numer_drogi,pikietaz,obszar,skrzyzowanie,geometria_drogi,charakterystyka_miejsca,oswietlenie,warunki_atmosferyczne,...,liczba_pasow,zachowanie_kierujacego,zachowanie_pieszego,inne_przyczyny,liczba_rannych,liczba_zabitych,liczba_kolizji,liczba_wypadkow,rok,miesiac,dzien_miesiąca,dzien_tygodnia,data,czy_weekend,handlowa
139849,2018-07-11,14,19*25'218,49*40'188,KOSZARAWA,NaN,439,P1429S,"10,4",Obszar zabudowany,NaN,"Zakręt, łuk",Jezdnia,Światło dzienne,Pochmurno,...,2.0,Inne przyczyny,NaN,NaN,0,0,1,0,2018,7,11,2,2018-07-11,0,0
158102,2018-10-30,15,18*57'279,49*58'362,PSZCZYNA,GÓRNOŚLĄSKA,11,I2,NaN,Obszar zabudowany,NaN,Odcinek prosty,"Parking, plac, MOP",Światło dzienne,Dobre warunki atmosferyczne,...,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018,10,30,1,2018-10-30,0,0


#### Współrzęne geograficzne

In [14]:
print('kolumna GPS X: ', df['GPS_x'].dtypes)
print('kolumna GPS Y: ', df['GPS_y'].dtypes)

kolumna GPS X:  object
kolumna GPS Y:  object


In [15]:
# Zamieńmy typ kolumn na float

#kod by Michal Gurgul
# https://stackoverflow.com/questions/33997361

def dms2dd(coord):
    """ GPS HH.MM.SS.S to DD (Decimal Degrees) conversion """
    c = re.findall(r"\w+", coord)
    degrees = float(c[0])
    minutes = float(c[1])
    if len(c) > 2:
        seconds = float(c[2])/10   #???? dlaczego dzielimy przez 10 - let's check it
    else:
        seconds = 0

    dd = float(degrees) + (minutes)/60 + float(seconds)/(60*60)

    return dd


df['GPS_x'] = df['GPS_x'].apply(lambda x : dms2dd(x))
df['GPS_y'] = df['GPS_y'].apply(lambda x : dms2dd(x)) 


#### Kolumny zawierające informacje o miejscu zdarzenia

In [16]:
print('kolumna miejscowosc : ', df['miejscowosc'].dtypes)
print('kolumna ulica : ',df['ulica'].dtypes)
print('kolumna numer_domu : ', df['numer_domu'].dtypes)
print('kolumna numer_drogi: ', df['numer_drogi'].dtypes)
print('Wszystko jest poprawnie.')

kolumna miejscowosc :  object
kolumna ulica :  object
kolumna numer_domu :  object
kolumna numer_drogi:  object
Wszystko jest poprawnie.


In [17]:
print('pikietaz: ', df['pikietaz'].dtypes)
print('Przykładowe wartości z tej kolumny: ', df['pikietaz'][df['pikietaz'].notnull()].sample(3))
print('To powinien być format FLOAT.')

pikietaz:  object
Przykładowe wartości z tej kolumny:  10167       0,9
82076     475,4
181726    300,2
Name: pikietaz, dtype: object
To powinien być format FLOAT.


In [18]:
# 1. Przekształćmy ',' na '.'

df['pikietaz'] = df['pikietaz'].str.replace(',','.')

# 2. Przekształćmy string na float

df['pikietaz'] = df['pikietaz'].astype(float)

print( 'Nowe wartości: ', df['pikietaz'][df['pikietaz'].notnull()].sample(3) )

Nowe wartości:  170751    294.0
208484    451.1
167549     22.2
Name: pikietaz, dtype: float64


In [19]:
print('kolumna obszar: ', df['obszar'].dtypes)

print('\nPrzykładowe wartości: ', df['obszar'].unique())

print('\nZamieńmy wartości na typ BOOLEAN - 1 oznaczające obszar zabudowany, 0 obszar niezabudowany')

df['obszar'] = df['obszar'].apply(lambda x: 1 if (x == 'Obszar zabudowany') else (0 if x == 'Obszar niezabudowany' else 'NaN'))

print('\nNowe wartości w kolumnie: ', df['obszar'].unique())

kolumna obszar:  object

Przykładowe wartości:  ['Obszar niezabudowany' 'Obszar zabudowany' nan]

Zamieńmy wartości na typ BOOLEAN - 1 oznaczające obszar zabudowany, 0 obszar niezabudowany

Nowe wartości w kolumnie:  [0 1 'NaN']


In [20]:
print('kolumna skrzyzowanie: ', df['skrzyzowanie'].dtypes)

print('\nPrzykładowe wartości: ', df['skrzyzowanie'].unique())

print('Ilość wartości nullowych:',df['skrzyzowanie'].isnull().sum().sum())

print('\nZamieńmy wartości na typ INT przyznając: \n 0 : O ruchu okrężnym \n 1 : Równorzędne \n 2 : Z drogą z pierwsz.')

df['Skrzyżowanie'] = df['skrzyzowanie'].apply(lambda x: 0 if x == 'O ruchu okrężnym' else (1 if x == 'Równorzędne' else (2 if x == 'Z drogą z pierwsz.' else 'NaN')))

print('\n Nowe wartości w kolumnie: ', df['skrzyzowanie'].unique())

kolumna skrzyzowanie:  object

Przykładowe wartości:  [nan 'Z drogą z pierwsz.' 'O ruchu okrężnym' 'Równorzędne']
Ilość wartości nullowych: 166779

Zamieńmy wartości na typ INT przyznając: 
 0 : O ruchu okrężnym 
 1 : Równorzędne 
 2 : Z drogą z pierwsz.

 Nowe wartości w kolumnie:  [nan 'Z drogą z pierwsz.' 'O ruchu okrężnym' 'Równorzędne']


In [21]:
df.sample(3)

,data_zdarzenia,godzina,GPS_x,GPS_y,miejscowosc,ulica,numer_domu,numer_drogi,pikietaz,obszar,skrzyzowanie,geometria_drogi,charakterystyka_miejsca,oswietlenie,warunki_atmosferyczne,...,zachowanie_kierujacego,zachowanie_pieszego,inne_przyczyny,liczba_rannych,liczba_zabitych,liczba_kolizji,liczba_wypadkow,rok,miesiac,dzien_miesiąca,dzien_tygodnia,data,czy_weekend,handlowa,Skrzyżowanie
18835,2016-05-22,12,18.558333,50.388639,PYSKOWICE,A.MICKIEWICZA,NaN,40,99.9,0,NaN,Odcinek prosty,Jezdnia,Światło dzienne,Dobre warunki atmosferyczne,...,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2016,5,22,6,2016-05-22,1,0,NaN
132562,2018-05-24,5,18.784778,49.926194,WISŁA MAŁA,NAD JEZIOREM,72,W939,NaN,1,NaN,"Zakręt, łuk",Jezdnia,Światło dzienne,Dobre warunki atmosferyczne,...,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2018,5,24,3,2018-05-24,0,0,NaN
179315,2019-03-07,18,19.385583,50.299306,SŁAWKÓW,ŚWIĘTOJAŃSKA,NaN,G665071S,NaN,1,Z drogą z pierwsz.,NaN,Jezdnia,Noc - droga nieoświetlona,Pochmurno,...,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2019,3,7,3,2019-03-07,0,0,2


In [22]:
print('kolumna geometria_drogi: ', df['geometria_drogi'].dtypes)

print('\nPrzykładowe wartości: ', df['geometria_drogi'].unique())

df['geometria_drogi'] = df['geometria_drogi'].str.replace(";","").str.replace(",","").str.replace(".","").str.lower()

print('\n Nowe wartości w kolumnie: ', df['geometria_drogi'].unique())

kolumna geometria_drogi:  object

Przykładowe wartości:  ['Odcinek prosty' nan 'Zakręt, łuk' 'Spadek' 'Wzniesienie'
 'Wierzchołek wzn.' 'Zakręt, łuk; Spadek' 'Odcinek prosty; Spadek'
 'Zakręt, łuk; Wzniesienie' 'Odcinek prosty; Wzniesienie'
 'Zakręt, łuk; Wierzchołek wzn.' 'Odcinek prosty; Wierzchołek wzn.'
 'Odcinek prosty; Zakręt, łuk']

 Nowe wartości w kolumnie:  ['odcinek prosty' nan 'zakręt łuk' 'spadek' 'wzniesienie'
 'wierzchołek wzn' 'zakręt łuk spadek' 'odcinek prosty spadek'
 'zakręt łuk wzniesienie' 'odcinek prosty wzniesienie'
 'zakręt łuk wierzchołek wzn' 'odcinek prosty wierzchołek wzn'
 'odcinek prosty zakręt łuk']


In [23]:
print('kolumna charakterystyka_miejsca: ', df['charakterystyka_miejsca'].dtypes)

print('\nPrzykładowe wartości: ', df['charakterystyka_miejsca'].unique())

df['charakterystyka_miejsca'] = df['charakterystyka_miejsca'].str.replace('(od 11.2015)','').str.replace(";","").str.replace(",","").str.replace(".","").str.replace("()","").str.lower()

#nie usuwa () - do poprawy

print('\n Nowe wartości w kolumnie: ', df['charakterystyka_miejsca'].unique())

kolumna charakterystyka_miejsca:  object

Przykładowe wartości:  ['Jezdnia' 'Chodnik, droga dla pieszych' 'Parking, plac'
 'Wjazd, wyjazd z posesji, pola' 'Parking, plac, MOP (od 11.2015)'
 'Skarpa, rów' 'Pobocze' 'Przejście dla pieszych'
 'Most, wiadukt, łącznica, tunel' 'Pas dzielący jezdnie'
 'Przejazd kolejowy strzeżony' 'Przejazd tramwajowy, torowisko'
 'Przystanek komunikacji publicznej' 'Droga dla rowerzystów'
 'Przejazd kolejowy niestrzeżony' 'Roboty drogowe, oznakowanie tymczasowe'
 'Droga, pas ruchu, śluza dla rowerów (od 11.2015)'
 'Przejazd dla rowerzystów (od 11.2015)'
 'Przewiązka na drodze dwujezdniowej' 'Parking, plac, MOP'
 'Przejazd tramwajowy, torowisko tramwajowe'
 'Droga, pas ruchu, śluza dla rowerów' 'Przejazd dla rowerzystów' nan]

 Nowe wartości w kolumnie:  ['jezdnia' 'chodnik droga dla pieszych' 'parking plac'
 'wjazd wyjazd z posesji pola' 'parking plac mop ()' 'skarpa rów'
 'pobocze' 'przejście dla pieszych' 'most wiadukt łącznica tunel'
 'pas dzielący jezdn

In [24]:
print('kolumna oswietlenie: ', df['oswietlenie'].dtypes)

print('\nPrzykładowe wartości: ', df['oswietlenie'].unique())

#Stwórzmy dodatkowe kolumny, które pozwolą sprawdzić, czy to był dzień czy noc, czy droga posiada oświetlenie

df['oswietlenie'] = df['oswietlenie'].apply(func = lambda x: 'Świt, zmrok' if x == 'Zmrok, świt' else x)

df['oswietlenie'] = df['oswietlenie'].apply(func = lambda x: 'Dzień' if x == 'Światło dzienne' else x)


kolumna oswietlenie:  object

Przykładowe wartości:  ['Noc - droga nieoświetlona' 'Noc - droga oświetlona' 'Światło dzienne'
 'Zmrok, świt' 'Świt, zmrok']


In [25]:
df['oswietlenie'].unique()

array(['Noc - droga nieoświetlona', 'Noc - droga oświetlona', 'Dzień',
       'Świt, zmrok'], dtype=object)

In [26]:
print('kolumna warunki_atmosferyczne: ', df['warunki_atmosferyczne'].dtypes)

print('\nPrzykładowe wartości: ', df['warunki_atmosferyczne'].unique())

#Proponuję stworzyć dodatkowe kolumny typu : bylo_slonce, byl_deszcz, etc

df['warunki_atmosferyczne'] = df['warunki_atmosferyczne'].str.lower().str.replace(",","").str.replace(";","").str.replace(",","")


#kod by Ania Cibis

df['pogoda_dobre_warunki']        = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'warunki' in str(x) else 0)
df['pogoda_pochmurno']            = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'pochmurno' in str(x) else 0)
df['pogoda_mgla']                 = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'mgła' in str(x) else 0)
df['pogoda_dym']                  = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'dym' in str(x) else 0)
df['pogoda_oslep_slonce']         = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'słońce' in str(x) else 0)
df['pogoda_opady']                = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'opady' in str(x) else 0)
df['pogoda_snieg']                = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'śniegu' in str(x) else 0)
df['pogoda_grad']                 = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'gradu' in str(x) else 0)
df['pogoda_deszcz']               = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'deszczu' in str(x) else 0)
df['pogoda_wiatr']                = df['warunki_atmosferyczne'].apply(lambda x: 1 if 'wiatr' in str(x) else 0)

df['warunki_atmosferyczne'].unique()

kolumna warunki_atmosferyczne:  object

Przykładowe wartości:  ['Dobre warunki atmosferyczne' 'Pochmurno' 'Mgła, dym'
 'Oślepiające słońce' 'Opady śniegu, gradu'
 'Pochmurno; Opady śniegu, gradu' 'Opady deszczu'
 'Opady deszczu; Opady śniegu, gradu' 'Pochmurno; Opady deszczu'
 'Pochmurno; Mgła, dym' 'Pochmurno; Opady deszczu; Mgła, dym'
 'Opady deszczu; Mgła, dym'
 'Pochmurno; Opady deszczu; Opady śniegu, gradu'
 'Silny wiatr; Pochmurno; Opady śniegu, gradu'
 'Pochmurno; Opady śniegu, gradu; Mgła, dym'
 'Opady śniegu, gradu; Mgła, dym' 'Silny wiatr'
 'Silny wiatr; Opady deszczu' 'Silny wiatr; Pochmurno'
 'Silny wiatr; Opady śniegu, gradu' 'Oślepiające słońce; Silny wiatr'
 'Silny wiatr; Pochmurno; Opady deszczu'
 'Oślepiające słońce; Opady deszczu' nan
 'Silny wiatr; Opady deszczu; Opady śniegu, gradu'
 'Silny wiatr; Pochmurno; Opady deszczu; Opady śniegu, gradu'
 'Opady deszczu; Opady śniegu, gradu; Mgła, dym']


array(['dobre warunki atmosferyczne', 'pochmurno', 'mgła dym',
       'oślepiające słońce', 'opady śniegu gradu',
       'pochmurno opady śniegu gradu', 'opady deszczu',
       'opady deszczu opady śniegu gradu', 'pochmurno opady deszczu',
       'pochmurno mgła dym', 'pochmurno opady deszczu mgła dym',
       'opady deszczu mgła dym',
       'pochmurno opady deszczu opady śniegu gradu',
       'silny wiatr pochmurno opady śniegu gradu',
       'pochmurno opady śniegu gradu mgła dym',
       'opady śniegu gradu mgła dym', 'silny wiatr',
       'silny wiatr opady deszczu', 'silny wiatr pochmurno',
       'silny wiatr opady śniegu gradu', 'oślepiające słońce silny wiatr',
       'silny wiatr pochmurno opady deszczu',
       'oślepiające słońce opady deszczu', nan,
       'silny wiatr opady deszczu opady śniegu gradu',
       'silny wiatr pochmurno opady deszczu opady śniegu gradu',
       'opady deszczu opady śniegu gradu mgła dym'], dtype=object)

#### Kolumny zawierające informacje o zdarzeniu

In [27]:
print('kolumna rodzaj_zdarzenia: ', df['rodzaj_zdarzenia'].dtypes)

print('\nPrzykładowe wartości: ', df['rodzaj_zdarzenia'].unique())

df['rodzaj_zdarzenia'] = df['rodzaj_zdarzenia'].str.replace(",","").str.replace(";","").str.replace(":","").str.replace(";","")

print('\n Nowe wartości w kolumnie: ', df['rodzaj_zdarzenia'].unique())

kolumna rodzaj_zdarzenia:  object

Przykładowe wartości:  ['Najechanie na zwierzę' 'Najechanie na pojazd unieruchomiony'
 'Zderzenie pojazdów tylne' 'Najechanie na słup, znak'
 'Zderzenie pojazdów boczne' 'Najechanie na barierę ochronną' 'Inne'
 'Najechanie na dziurę, wybój, garb' 'Najechanie na pieszego'
 'Wywrócenie się pojazdu' 'Zderzenie pojazdów czołowe'
 'Najechanie na drzewo' 'Najechanie na zapore kolejową'
 'Zdarzenie z pasażerem']

 Nowe wartości w kolumnie:  ['Najechanie na zwierzę' 'Najechanie na pojazd unieruchomiony'
 'Zderzenie pojazdów tylne' 'Najechanie na słup znak'
 'Zderzenie pojazdów boczne' 'Najechanie na barierę ochronną' 'Inne'
 'Najechanie na dziurę wybój garb' 'Najechanie na pieszego'
 'Wywrócenie się pojazdu' 'Zderzenie pojazdów czołowe'
 'Najechanie na drzewo' 'Najechanie na zapore kolejową'
 'Zdarzenie z pasażerem']


In [28]:
print('kolumna wina: ', df['wina'].dtypes)

print('\nPrzykładowe wartości: ', df['wina'].unique())

kolumna wina:  object

Przykładowe wartości:  ['z innych przyczyn' 'z winy kierującego' 'współwina uczestników ruchu'
 'z winy pieszego']


In [29]:
print('kolumna rodzaj_drogi: ', df['rodzaj_drogi'].dtypes)

print('\nPrzykładowe wartości: ', df['rodzaj_drogi'].unique())

df['rodzaj_drogi'] = df['rodzaj_drogi'].str.lower()

print('\nNowe wartości w kolumnie: ', df['rodzaj_drogi'].unique())

kolumna rodzaj_drogi:  object

Przykładowe wartości:  ['Jednojezdniowa dwukierunkowa' 'Jednokierunkowa' 'Autostrada'
 'Dwie jezdnie jednokierunkowe' 'Ekspresowa' nan]

Nowe wartości w kolumnie:  ['jednojezdniowa dwukierunkowa' 'jednokierunkowa' 'autostrada'
 'dwie jezdnie jednokierunkowe' 'ekspresowa' nan]


In [30]:
print('kolumna nawierzchnia_drogi: ', df['nawierzchnia_drogi'].dtypes)

print('\nPrzykładowe wartości: ', df['nawierzchnia_drogi'].unique())

df['nawierzchnia_drogi'] = df['nawierzchnia_drogi'].str.lower()

df.rename(columns={'nawierzchnia_drogi':'nawierzchnia_twarda'}, inplace=True) 

#zamieńmy wartości na zmienne 0 i 1

df['nawierzchnia_twarda'] = df['nawierzchnia_twarda'].apply(lambda x: 1 if x == 'Twarda' else (0 if x == 'Gruntowa' else 'NaN'))

print('\nNowe wartości w kolumnie: ', df['nawierzchnia_twarda'].unique())

kolumna nawierzchnia_drogi:  object

Przykładowe wartości:  ['Twarda' 'Gruntowa' nan]

Nowe wartości w kolumnie:  ['NaN']


In [31]:
print('kolumna stan_nawierzchni: ',df['stan_nawierzchni'].dtypes)

df['stan_nawierzchni'] = df['stan_nawierzchni'].str.replace(",","").str.replace(";","").str.replace(":","").str.lower()

print('\nPrzykładowe wartości: ',df['stan_nawierzchni'].unique())

kolumna stan_nawierzchni:  object

Przykładowe wartości:  ['sucha' 'mokra' 'oblodzona zaśnieżona' 'mokra oblodzona zaśnieżona'
 'koleiny garby' 'zanieczyszczona' 'mokra koleiny garby'
 'oblodzona zaśnieżona dziury wyboje' 'mokra dziury wyboje'
 'kałuże rozlewiska' 'kałuże rozlewiska oblodzona zaśnieżona'
 'mokra kałuże rozlewiska' 'zanieczyszczona dziury wyboje' 'dziury wyboje'
 'mokra kałuże rozlewiska dziury wyboje' 'sucha dziury wyboje'
 'sucha zanieczyszczona' 'mokra kałuże rozlewiska koleiny garby'
 'mokra zanieczyszczona'
 'mokra zanieczyszczona koleiny garby dziury wyboje'
 'mokra kałuże rozlewiska oblodzona zaśnieżona'
 'mokra koleiny garby dziury wyboje' 'kałuże rozlewiska dziury wyboje'
 'oblodzona zaśnieżona zanieczyszczona koleiny garby dziury wyboje'
 'zanieczyszczona koleiny garby dziury wyboje'
 'mokra kałuże rozlewiska zanieczyszczona koleiny garby dziury wyboje'
 'kałuże rozlewiska zanieczyszczona dziury wyboje' 'sucha koleiny garby'
 'sucha koleiny garby dziury wyboje

In [32]:
#Powtórzmy proces przeprowadzony dla pogody i spróbujmy stworzyć dodatkowe kolumny dla nawierzchni

df['nawierzchnia_sucha'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'sucha' in str(x) else 0)
df['nawierzchnia_mokra'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'mokra' in str(x) else 0)
df['nawierzchnia_oblodzona'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'oblodzona' in str(x) else 0)
df['nawierzchnia_zasniezona'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'zaśnieżona' in str(x) else 0)
df['nawierzchnia_nierowna'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'wyboje' in str(x) or 'dziury' in str(x) or 'koleiny' in str(x) or 'garby' in str(x) else 0)
df['nawierzchnia_kaluze'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'kałuże' in str(x) or 'rozlewiska' in str(x) else 0)
df['nawierzchnia_zanieszyszczenia'] = df['stan_nawierzchni'].apply(lambda x: 1 if 'zanieczyszczenia' in str(x) else 0)



In [33]:
print('kolumna sygnalizacja: ', df['sygnalizacja'].dtypes)

print('\nPrzykładowe wartości: ', df['sygnalizacja'].unique())

#zamieńmy wartości na zmienne 0 i 1 (sygnalizacja jest/nie ma)

df['sygnalizacja'] = df['sygnalizacja'].apply(lambda x: 1 if x == 'Jest, działa' else (0 if x =='Brak' else -1 ))


print('\nNowe wartości w kolumnie sygnalizacja: ', df['sygnalizacja'].unique())

kolumna sygnalizacja:  object

Przykładowe wartości:  ['Brak' 'Jest, nie działa' 'Jest, działa']

Nowe wartości w kolumnie sygnalizacja:  [ 0 -1  1]


In [34]:
print('kolumna oznakowanie_poziome: ', df['oznakowanie_poziome'].dtypes)
print('\nPrzykładowe wartości: ', df['oznakowanie_poziome'].unique())

#zamieńmy na 0 i 1 (oznakowanie jest/nie ma)

df['oznakowanie_poziome'] = df['oznakowanie_poziome'].apply(lambda x: 0 if x == 'Jest' else (1 if x == 'Nie ma' else None))

print('\nNowe wartości w kolumnie oznakowanie_poziome: ',df['oznakowanie_poziome'].unique())

kolumna oznakowanie_poziome:  object

Przykładowe wartości:  ['Jest' 'Nie ma' nan]

Nowe wartości w kolumnie oznakowanie_poziome:  [ 0.  1. nan]


In [35]:
print('kolumna liczba_pasow: ', df['liczba_pasow'].dtypes)
print('\nPrzykładowe wartości: ', df['liczba_pasow'].unique())

df['liczba_pasow'].value_counts().head(10)

#liczba pasów - do dalszej analizy...

kolumna liczba_pasow:  float64

Przykładowe wartości:  [  2.  nan   1.   3.   4.   5.   6.   8.   7.   0.   9. 250.  50.  24.
  90.  12.  60.  20.  70.  40.  18.  13.  10.  14.  17.  66.  11.  16.
  25.  22.  32.  30.  67.  15.]


2.0     101699
4.0      14937
3.0       9186
1.0       8090
6.0       3577
5.0       2537
8.0        671
7.0        433
10.0       166
12.0       131
Name: liczba_pasow, dtype: int64

In [36]:
print('kolumna zachowanie_kierujacego: ', df['zachowanie_kierujacego'].dtypes)
print('\nPrzykładowe wartości: ', df['zachowanie_kierujacego'].unique())
df['zachowanie_kierujacego'] = df['zachowanie_kierujacego'].str.replace(':','').str.replace('(od 11.2015)','').str.replace(';','').str.replace(',','').str.replace('.','').str.lower()
print('\nPrzykładowe wartości: ', df['zachowanie_kierujacego'].unique())

kolumna zachowanie_kierujacego:  object

Przykładowe wartości:  [nan 'Nieprawidłowe: cofanie' 'Nieprawidłowe: omijanie'
 'Niedostosowanie prędkości do warunków ruchu'
 'Nieudzielenie pierwszeństwa przejazdu' 'Inne przyczyny (od 11.2015)'
 'Niezachowanie bezp. odl. między pojazdami' 'Nieprawidłowe: skręcanie'
 'Nieprawidłowe: zmienianie pasa ruchu' 'Zmęczenie, zaśnięcie'
 'Nieprawidłowe: wymijanie' 'Nieprzestrzeganie innych sygnałów'
 'Niestosowanie się do sygnalizacji świetlnej (od 11.2015)'
 'Nieustąpienie pierwszeństwa pieszemu na przejściu dla pieszych (od 11.2015)'
 'Gwałtowne hamowanie' 'Nieprawidłowe: wyprzedzanie'
 'Nieprawidłowe: zatrzymywanie, postój'
 'Nieprawidłowe: wymijanie; Nieprawidłowe: wymijanie'
 'Nieprawidłowe: cofanie; Nieprawidłowe: cofanie'
 'Nieprawidłowe: Zawracanie'
 'Nieprawidłowe: skręcanie; Nieprawidłowe: skręcanie'
 'Nieustąpienie pierwszeństwa pieszemu w innych okolicznościach (od 11.2015)'
 'Nieustąpienie pierwszeństwa pieszemu przy skręcaniu w drogę popr


Przykładowe wartości:  [nan 'nieprawidłowe cofanie' 'nieprawidłowe omijanie'
 'niedostosowanie prędkości do warunków ruchu'
 'nieudzielenie pierwszeństwa przejazdu' 'inne przyczyny ()'
 'niezachowanie bezp odl między pojazdami' 'nieprawidłowe skręcanie'
 'nieprawidłowe zmienianie pasa ruchu' 'zmęczenie zaśnięcie'
 'nieprawidłowe wymijanie' 'nieprzestrzeganie innych sygnałów'
 'niestosowanie się do sygnalizacji świetlnej ()'
 'nieustąpienie pierwszeństwa pieszemu na przejściu dla pieszych ()'
 'gwałtowne hamowanie' 'nieprawidłowe wyprzedzanie'
 'nieprawidłowe zatrzymywanie postój'
 'nieprawidłowe wymijanie nieprawidłowe wymijanie'
 'nieprawidłowe cofanie nieprawidłowe cofanie' 'nieprawidłowe zawracanie'
 'nieprawidłowe skręcanie nieprawidłowe skręcanie'
 'nieustąpienie pierwszeństwa pieszemu w innych okolicznościach ()'
 'nieustąpienie pierwszeństwa pieszemu przy skręcaniu w drogę poprzeczną  ()'
 'niedostosowanie prędkości do warunków ruchu niedostosowanie prędkości do warunków ruchu'

In [37]:
#kod by Anna Cibis
#teraz spróbujmy stworzyć dodatkowe kolumny dla zachowania kierującego

df['zachowanie_kierujacego_niepr_cofanie']             = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'cofanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_omijanie']            = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'omijanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niedost_predkosci']         = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'prędkości' in str(x).lower() else 0)
df['zachowanie_kierujacego_nieudzielenie_pierw']       = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'pierwszeństwa' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_skrecanie']           = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'skręcanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niezachowanie_odl']         = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'bezp. odl.' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_zmiana_pasa']         = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'zmienianie ' in str(x).lower() else 0)
df['zachowanie_kierujacego_zmeczenie']                 = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'zmęczenie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_wymijanie']           = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'wymijanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_nieprzestrz_znakow']        = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'nieprzestrzeganie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niestos_do_sygn']           = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'sygnalizacji' in str(x).lower() else 0)
df['zachowanie_kierujacego_nieust_przejazdu']          = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'przejazdu' in str(x).lower() else 0)
df['zachowanie_kierujacego_nieust_pieszemu']           = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'pieszemu' in str(x).lower() else 0)
df['zachowanie_kierujacego_gwalt_hamowanie']           = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'gwałtowne' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_wyprzedzanie']        = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'wyprzedzanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_zatrzymanie']         = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'zatrzymanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_zawracanie']          = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'zawracanie' in str(x).lower() else 0)
df['zachowanie_kierujacego_niepr_przejazd_rowerowy']   = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'rowerzystów' in str(x).lower() else 0)
df['zachowanie_kierujacego_jazda_bez_swiatel']         = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'świateł' in str(x).lower() else 0)
df['zachowanie_kierujacego_inne_przyczyny']            = df['zachowanie_kierujacego'].apply(lambda x: 1 if 'inne' in str(x).lower() else 0)

In [38]:
print('kolumna inne_przyczyny: ', df['inne_przyczyny'].dtypes)
print('\nPrzykładowe wartości: ', df['inne_przyczyny'].unique())

kolumna inne_przyczyny:  object

Przykładowe wartości:  ['Obiekty, zwierzęta na drodze' nan 'Nieustalone'
 'Niewłaściwy stan jezdni' 'Inne'
 'Utrata przytomności, śmierć kierującego (od 11.2015)' 'Pożar pojazdu'
 'Nieprawidłowo zabezp. roboty drogowe'
 'Niesprawność techniczna pojazdu (od 11.2015)' 'Z winy pasażera'
 'Oślepienie przez inny pojazd lub słońce'
 'Nieprawidłowo działająca zapora, rogatka' 'Stan jezdni'
 'Niesprawność techniczna pojazdu' 'Zabezpieczenie robót na drodze'
 'Utrata przytomności, śmierć kierującego' 'Organizacja ruchu'
 'Nieprawidłowo działająca zapora kolej.'
 'Działanie sygnalizacji świetlnej']


In [39]:
print('kolumna zachowanie_pieszego: ', df['zachowanie_pieszego'].dtypes)
print('\nPrzykładowe wartości: ', df['zachowanie_pieszego'].unique())
df['zachowanie_pieszego'] = df['zachowanie_pieszego'].str.replace('(od 11.2015)','').str.replace(':','').str.replace(';','').str.replace(',','').str.replace('.','').str.lower()


df['zachowanie_pieszego'].unique()



kolumna zachowanie_pieszego:  object

Przykładowe wartości:  [nan 'Wejście na jezdnię przy czerwonym świetle'
 'Przekraczanie jezdni w miejscu niedozwolonym'
 'Nieostrożne wejście na jezdnię: przed jadącym pojazdem'
 'Inne przyczyny (od 11.2015)'
 'Nieostrożne wejście na jezdnię: zza pojazdu, przeszkody'
 'Chodzenie nieprawidłową stroną drogi' 'Stanie na jezdni, leżenie'
 'Nieostrożne wejście na jezdnię: przed jadącym pojazdem; Nieostrożne wejście na jezdnię: przed jadącym pojazdem'
 'Zatrzymanie, cofnięcie się'
 'Przekraczanie jezdni w miejscu niedozwolonym; Przekraczanie jezdni w miejscu niedozwolonym'
 'Inne przyczyny' 'Leżenie, siedzenie, klęczenie, stanie na jezdni'
 'Nieostrożne wejście na jezdnię: zza pojazdu, przeszkody; Nieostrożne wejście na jezdnię: zza pojazdu, przeszkody'
 'Wejście na jezdnię przy czerwonym świetle; Wejście na jezdnię przy czerwonym świetle'
 'Inne przyczyny; Inne przyczyny'
 'Chodzenie nieprawidłową stroną drogi; Chodzenie nieprawidłową stroną drogi'
 'Le

array([nan, 'wejście na jezdnię przy czerwonym świetle',
       'przekraczanie jezdni w miejscu niedozwolonym',
       'nieostrożne wejście na jezdnię przed jadącym pojazdem',
       'inne przyczyny ()',
       'nieostrożne wejście na jezdnię zza pojazdu przeszkody',
       'chodzenie nieprawidłową stroną drogi', 'stanie na jezdni leżenie',
       'nieostrożne wejście na jezdnię przed jadącym pojazdem nieostrożne wejście na jezdnię przed jadącym pojazdem',
       'zatrzymanie cofnięcie się',
       'przekraczanie jezdni w miejscu niedozwolonym przekraczanie jezdni w miejscu niedozwolonym',
       'inne przyczyny', 'leżenie siedzenie klęczenie stanie na jezdni',
       'nieostrożne wejście na jezdnię zza pojazdu przeszkody nieostrożne wejście na jezdnię zza pojazdu przeszkody',
       'wejście na jezdnię przy czerwonym świetle wejście na jezdnię przy czerwonym świetle',
       'inne przyczyny inne przyczyny',
       'chodzenie nieprawidłową stroną drogi chodzenie nieprawidłową stroną dr

In [42]:
def simplify(a):        
    a = a.replace('\'','')                   # usuń znaki '
    a = a.replace('\n','')                   # usuń znaki nowej linii
    a = a.replace(';','')                    # usuń średniki
    a = a.replace(',','')                    # usuń przecinki
    a = a.replace('.',' ')                   # zamień kropki na spacje (ze względu na dziwny zapis w adresach)
    a = a.replace('  ',' ')                  # usuń podwójne spacje
    a = a.replace('(od 11 2015)','')         # usuń ciąg "(od 11.2015)"
    a = a.lower()                            # zamień wielkie litery na małe
    a=' '.join(unique_list(a.split()))       # usuń zduplikowane słowa
    return(a)

def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist    
    
def simplify_column(serie):
    a = serie.unique()                       # wrzuć wartości do tablicy numpy
    a = np.array2string(a)                   # przekształć tablicę na string
    return(simplify(a))

def simplify_cell(string):
    return(simplify(str(string)))

df['zachowanie_pieszego'] = df['zachowanie_pieszego'].apply(lambda x: simplify_cell(x))
df['zachowanie_pieszego'].unique()

array(['nan', 'wejście na jezdnię przy czerwonym świetle',
       'przekraczanie jezdni w miejscu niedozwolonym',
       'nieostrożne wejście na jezdnię przed jadącym pojazdem',
       'inne przyczyny ()',
       'nieostrożne wejście na jezdnię zza pojazdu przeszkody',
       'chodzenie nieprawidłową stroną drogi', 'stanie na jezdni leżenie',
       'zatrzymanie cofnięcie się', 'inne przyczyny',
       'leżenie siedzenie klęczenie stanie na jezdni'], dtype=object)

In [43]:
print('kolumna liczba_rannych: ', df['liczba_rannych'].dtypes)
print('\nPrzykładowe wartości: ', df['liczba_rannych'].unique())

kolumna liczba_rannych:  int64

Przykładowe wartości:  [ 0  4  1  2  3  6  5  9 11  8 26  7 10 12]


In [44]:
print('kolumna liczba_zabitych: ', df['liczba_zabitych'].dtypes)
print('\nPrzykładowe wartości: ', df['liczba_zabitych'].unique())

kolumna liczba_zabitych:  int64

Przykładowe wartości:  [0 1 2 4 3]


In [45]:
print('kolumna liczba_kolizji: ', df['liczba_kolizji'].dtypes)
print('\nPrzykładowe wartości: ', df['liczba_kolizji'].unique())

kolumna liczba_kolizji:  int64

Przykładowe wartości:  [1 0]


In [46]:
print('kolumna liczba_wypadkow: ', df['liczba_wypadkow'].dtypes)
print('\nPrzykładowe wartości: ', df['liczba_wypadkow'].unique())

kolumna liczba_wypadkow:  int64

Przykładowe wartości:  [0 1]


## KONIEC!

In [47]:
# No dobra, wcale nie ;) Stwórzmy nowy dataset, który zawiera wszystkie nasze istotne kolumny
# Istotne, czyli które?
cols = df.columns.tolist()
cols

['data_zdarzenia',
 'godzina',
 'GPS_x',
 'GPS_y',
 'miejscowosc',
 'ulica',
 'numer_domu',
 'numer_drogi',
 'pikietaz',
 'obszar',
 'skrzyzowanie',
 'geometria_drogi',
 'charakterystyka_miejsca',
 'oswietlenie',
 'warunki_atmosferyczne',
 'rodzaj_zdarzenia',
 'wina',
 'rodzaj_drogi',
 'nawierzchnia_twarda',
 'stan_nawierzchni',
 'sygnalizacja',
 'oznakowanie_poziome',
 'liczba_pasow',
 'zachowanie_kierujacego',
 'zachowanie_pieszego',
 'inne_przyczyny',
 'liczba_rannych',
 'liczba_zabitych',
 'liczba_kolizji',
 'liczba_wypadkow',
 'rok',
 'miesiac',
 'dzien_miesiąca',
 'dzien_tygodnia',
 'data',
 'czy_weekend',
 'handlowa',
 'Skrzyżowanie',
 'pogoda_dobre_warunki',
 'pogoda_pochmurno',
 'pogoda_mgla',
 'pogoda_dym',
 'pogoda_oslep_slonce',
 'pogoda_opady',
 'pogoda_snieg',
 'pogoda_grad',
 'pogoda_deszcz',
 'pogoda_wiatr',
 'nawierzchnia_sucha',
 'nawierzchnia_mokra',
 'nawierzchnia_oblodzona',
 'nawierzchnia_zasniezona',
 'nawierzchnia_nierowna',
 'nawierzchnia_kaluze',
 'nawierzchni

In [49]:
potrzebne = ['data_zdarzenia',
 'godzina',
 'GPS_x',
 'GPS_y',
 'miejscowosc',
 'ulica',
 'numer_domu',
 'numer_drogi',
 'pikietaz',
 'obszar',
 'skrzyzowanie',
 'geometria_drogi',
 'charakterystyka_miejsca',
 'oswietlenie',
 'rodzaj_zdarzenia',
 'wina',
 'rodzaj_drogi',
 'nawierzchnia_twarda',
 'stan_nawierzchni',
 'sygnalizacja',
 'oznakowanie_poziome',
 'liczba_pasow',
 'zachowanie_kierujacego',
 'zachowanie_pieszego',
 'inne_przyczyny',
 'liczba_rannych',
 'liczba_zabitych',
 'liczba_kolizji',
 'liczba_wypadkow',
 'rok',
 'miesiac',
 'dzien_miesiąca',
 'dzien_tygodnia',
 'data',
 'czy_weekend',
 'handlowa',
 'Skrzyżowanie',
 'pogoda_dobre_warunki',
 'pogoda_pochmurno',
 'pogoda_mgla',
 'pogoda_dym',
 'pogoda_oslep_slonce',
 'pogoda_opady',
 'pogoda_snieg',
 'pogoda_grad',
 'pogoda_deszcz',
 'pogoda_wiatr',
 'nawierzchnia_sucha',
 'nawierzchnia_mokra',
 'nawierzchnia_oblodzona',
 'nawierzchnia_zasniezona',
 'nawierzchnia_nierowna',
 'nawierzchnia_kaluze',
 'nawierzchnia_zanieszyszczenia',
 'zachowanie_kierujacego_niepr_cofanie',
 'zachowanie_kierujacego_niepr_omijanie',
 'zachowanie_kierujacego_niedost_predkosci',
 'zachowanie_kierujacego_nieudzielenie_pierw',
 'zachowanie_kierujacego_niepr_skrecanie',
 'zachowanie_kierujacego_niezachowanie_odl',
 'zachowanie_kierujacego_niepr_zmiana_pasa',
 'zachowanie_kierujacego_zmeczenie',
 'zachowanie_kierujacego_niepr_wymijanie',
 'zachowanie_kierujacego_nieprzestrz_znakow',
 'zachowanie_kierujacego_niestos_do_sygn',
 'zachowanie_kierujacego_nieust_przejazdu',
 'zachowanie_kierujacego_nieust_pieszemu',
 'zachowanie_kierujacego_gwalt_hamowanie',
 'zachowanie_kierujacego_niepr_wyprzedzanie',
 'zachowanie_kierujacego_niepr_zatrzymanie',
 'zachowanie_kierujacego_niepr_zawracanie',
 'zachowanie_kierujacego_niepr_przejazd_rowerowy',
 'zachowanie_kierujacego_jazda_bez_swiatel',
 'zachowanie_kierujacego_inne_przyczyny']

In [50]:
df_new = df.filter(potrzebne, axis=1).copy()

In [51]:
df_new.shape

(229929, 74)

In [54]:
#Zapiszmy nasz nowy dataset ;)
df_new.to_csv('Data/df_enriched.csv')